In [ ]:
# default_exp entry_point

In [ ]:
#export
import os
import argparse
import time
import importlib
import inspect
import logging
from gcp_runner.ai_platform_constants import DistributionStrategyType

def get_distribution_strategy_instance(distribution_strategy_type:DistributionStrategyType):
    import tensorflow as tf

    logging.info('initializing distribution strategy: %s', distribution_strategy_type)
    if distribution_strategy_type == DistributionStrategyType.TPU_STRATEGY:
        tpu = None
        try:
            logging.info('resolving to TPU cluster')
            tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
            logging.info('connecting to TPU cluster')
            tf.config.experimental_connect_to_cluster(tpu)
        except ValueError as e:
            logging.info('error connecting to TPU cluster: %s', e)
            return None
        logging.info('initializing TPU system')
        tf.tpu.experimental.initialize_tpu_system(tpu)
        distribution_strategy = tf.distribute.experimental.TPUStrategy(tpu)
        logging.info(
            'training using TPUStrategy, tpu.cluster_spec: %s', tpu.cluster_spec())
        return distribution_strategy
    elif args.distribution_strategy == DistributionStrategyType.ONE_DEVICE_STRATEGY:
        return tf.distribute.OneDeviceStrategy(device="/cpu:0")
    elif args.distribution_strategy:
        return eval(args.distribution_strategy)()
    
def parse_unknown_args(unknown_args):
    kwargs = {}
    for arg in unknown_args:
        parse_result = re.search('--(.+)=(.+)', arg, re.IGNORECASE)
        if parse_result is not None and parse_result.groups() is not None and len(parse_result.groups()) == 2:
            kwargs[parse_result.group(1).replace('-', '_')] = parse_result.group(2)
        else:
            print("can't parse argument: %s" % arg)
    return kwargs

def main():
    print('in gcp_runner entry point')
    args_parser = argparse.ArgumentParser()
    args_parser.add_argument(
        '--module-name',
        help='module name of an app to run',
        required=True)
    args_parser.add_argument(
        '--function-name',
        help='function name to run',
        required=True)
    args_parser.add_argument(
        '--distribution-strategy',
        help='distribution strategy',
        choices=list([e.value for e in DistributionStrategyType]))
    
    args, unknown_args = args_parser.parse_known_args()
    print('running entrypoint function: %s.%s' % (args.module_name, args.function_name))
    module = importlib.import_module(args.module_name)
    func = getattr(module, args.function_name)
    if unknown_args is None or len(unknown_args) == 0:
        return func()
    
    print('additional args: %s' % str(unknown_args))
    args_spec = inspect.getargspec(func)
    if args_spec is None or args_spec.keywords is None or len(args_spec.keywords) == 0:
        print('provided function does not take any arguments, running as is')
        return func()
        
    distribution_strategy = None    
    if (args.distribution_strategy is not None):
        distribution_strategy_type = DistributionStrategyType(args.distribution_strategy)
        distribution_strategy = get_distribution_strategy_instance(distribution_strategy_type)
        
    kwargs = parse_unknown_args(unknown_args)
    if distribution_strategy is not None:
        kwargs['distribution_strategy'] = distribution_strategy
    
    return func(kwargs)
    
if __name__ == '__main__':
    main()

in gcp_runner entry point


usage: ipykernel_launcher.py [-h] --module-name MODULE_NAME --function-name
                             FUNCTION_NAME
ipykernel_launcher.py: error: argument --module-name is required


SystemExit: 2

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from nbdev.export import *
notebook2script()

ImportError: No module named nbdev.export